In [87]:
import os 
import sys
import tensorflow as tf
import numpy as np

sys.path.append("../")

from src.data.make_dataset import INDEX_SHEET_NAME,load_data
from src.features.build_features import generate_features
from src.models.LSTM import build_lstm_model, generate_train_val_data


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
INDEX_SHEET_NAME

['HangSeng Index Data',
 'S&P500 Index Data',
 'CSI300 Index Data',
 'DJIA index Data',
 'Nikkei 225 index Data',
 'Nifty 50 index Data']

In [15]:
raw = load_data(sheet_name=INDEX_SHEET_NAME[0])
raw.head()

,Ntime,Time,Closing Price,Open Price,High price,Low Price,Volume,MACD,CCI,ATR,...,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,HIBOR
0,20080702,733591,21704.4492,21785.3906,21938.1992,21555.5293,254858.26,-646.053200,-193.115118,546.4805,...,22956.709411,22497.96286,-751.5703,22187.92768,-887.8516,-5.754486,-0.069509,-401619.369837,71.99,1.6
1,20080703,733592,21242.7793,21389.4902,21742.0703,21163.5703,272528.48,-702.282620,-186.955957,578.5000,...,22793.477972,22289.66071,-1392.3806,21909.45156,-1786.9102,-7.872373,-0.068201,-507305.257079,72.73,1.6
2,20080704,733593,21423.8203,21402.1699,21534.0508,21344.8496,179634.42,-723.891770,-133.742667,291.2715,...,22663.034384,22152.28180,-1031.8496,21703.08164,-1634.1699,-8.153977,-0.058111,-398848.614015,72.71,1.6
3,20080707,733596,21913.0605,21402.6992,21916.2109,21402.6992,196457.92,-693.544830,-77.445777,513.5117,...,22591.608300,22069.02789,-129.2891,21677.22382,-1412.7403,-3.880007,-0.035303,-264228.169678,72.71,1.6
4,20080708,733597,21220.8105,21632.6992,21684.2109,21098.8398,201519.68,-717.087314,-108.634123,814.2207,...,22461.056128,21919.61285,-881.1993,21500.98396,-1576.7989,-6.703666,-0.055006,-518512.844172,72.96,1.6


In [120]:
generate_features(raw, sheet_name=INDEX_SHEET_NAME[0])

>>>>201010 finished!<<<<
>>>>201101 finished!<<<<
>>>>201104 finished!<<<<
>>>>201107 finished!<<<<
>>>>201110 finished!<<<<
>>>>201201 finished!<<<<
>>>>201204 finished!<<<<
>>>>201207 finished!<<<<
>>>>201210 finished!<<<<
>>>>201301 finished!<<<<
>>>>201304 finished!<<<<
>>>>201307 finished!<<<<
>>>>201310 finished!<<<<
>>>>201401 finished!<<<<
>>>>201404 finished!<<<<
>>>>201407 finished!<<<<
>>>>201410 finished!<<<<
>>>>201501 finished!<<<<
>>>>201504 finished!<<<<
>>>>201507 finished!<<<<
>>>>201510 finished!<<<<
>>>>201601 finished!<<<<
>>>>201604 finished!<<<<
>>>>201607 finished!<<<<
>>>> Feature generation complete! <<<<


### LSTM

In [121]:
result_dict = dict()
EPOCHS=60# 00
past_history = 4

In [ ]:
for index in INDEX_SHEET_NAME:
    result_dict[index]=dict()
    data_dir = f'../data/processed/wsae/{index}'
    train_lst = os.listdir(data_dir)
    for name in train_lst:
        x_train = np.load(data_dir+f'/{name}/X_train.npy')
        y_train = np.load(data_dir+f'/{name}/Y_train.npy')
        x_val = np.load(data_dir+f'/{name}/X_val.npy')
        y_val = np.load(data_dir+f'/{name}/Y_val.npy')
        x_test = np.load(data_dir+f'/{name}/X_test.npy')
        y_test = np.load(data_dir+f'/{name}/Y_test.npy')
        
        
        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history =4,batch_size=60
        )
        
        lstm = build_lstm_model(inputs_shape=[4,10], 
                        layers=5, 
                        units=[64, 64, 64, 64, 64], 
                        learning_rate=0.05)
        lstm.fit(train_data,
                 epochs=EPOCHS,
                 validation_data=val_data,
                 verbose=0)
        
        result_dict[index][name] = lstm.evaluate(test_data)
        
        save_dir = f'../models/{index}/{name}'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        lstm.save(save_dir+'/wase-lstmt.h5')
        print(f">>>>{index} {name} done!<<<<")

1/1 [==============================] - 0s 17ms/step - loss: 0.0326 - mean_absolute_percentage_error: 17.0551 - mape: 0.1706 - r: 0.7169 - theil_u: 0.0963
>>>>HangSeng Index Data 201010 done!<<<<
1/1 [==============================] - 0s 15ms/step - loss: 0.0255 - mean_absolute_percentage_error: 14.7370 - mape: 0.1474 - r: -0.2444 - theil_u: 0.0833
>>>>HangSeng Index Data 201101 done!<<<<
1/1 [==============================] - 0s 15ms/step - loss: 5.9859e-04 - mean_absolute_percentage_error: 2.3242 - mape: 0.0232 - r: 0.9336 - theil_u: 0.0142
>>>>HangSeng Index Data 201104 done!<<<<


In [110]:
x_val.shape, y_val.shape

((62, 10), (62,))

In [111]:
x_test.shape

(60, 10)

In [112]:
y_test.shape

(59,)

In [2]:
import numpy as np
index = 'HangSeng Index Data'
data_dir = f'../data/processed/wsae/{index}'
np.load(data_dir+f'/{201107}/X_train.npy').shape[0]//60

8

In [114]:
name

'201104'